In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_path = '/content/drive/MyDrive/Deepseek' 

In [ ]:
!pip install -q unsloth # install unsloth
!pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q -U datasets

In [ ]:
from unsloth import FastLanguageModel
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
ds = load_dataset("Magpie-Align/Magpie-Reasoning-V2-250K-CoT-Deepseek-R1-Llama-70B")

README.md:   0%|          | 0.00/7.65k [00:00<?, ?B/s]

train-00000-of-00009.parquet:   0%|          | 0.00/167M [00:00<?, ?B/s]

train-00001-of-00009.parquet:   0%|          | 0.00/173M [00:00<?, ?B/s]

train-00002-of-00009.parquet:   0%|          | 0.00/180M [00:00<?, ?B/s]

train-00003-of-00009.parquet:   0%|          | 0.00/184M [00:00<?, ?B/s]

...

train-00005-of-00009.parquet:   0%|          | 0.00/180M [00:00<?, ?B/s]

train-00006-of-00009.parquet:   0%|          | 0.00/172M [00:00<?, ?B/s]

train-00007-of-00009.parquet:   0%|          | 0.00/180M [00:00<?, ?B/s]

train-00008-of-00009.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/249922 [00:00<?, ? examples/s]

In [ ]:
filtered_ds = ds['train'].filter(
    lambda x: (
        x['input_quality'] == 'excellent' and
        x['task_category'] == 'Math' and
        '</think>' in x['response'] and
        all(x[field] is not None for field in x.keys())
    )
)
columns_to_keep = ['instruction', 'response', 'intent', 'knowledge', 'difficulty']
filtered_ds = filtered_ds.select_columns(columns_to_keep)
ds.cleanup_cache_files()
del ds

Filter:   0%|          | 0/249922 [00:00<?, ? examples/s]

In [ ]:
train_prompt_style = """Below is an instruction that describes a mathematical task, paired with additional context information to guide the solution.
Write a response that thoroughly solves the given problem.
Before solving, develop a clear step-by-step chain of reasoning to ensure accuracy and logical coherence.

### Instruction:
You are a mathematics expert with advanced knowledge in mathematical reasoning, problem-solving, and proof techniques. You think outloud and consider various aspects before giving any concrete answers.

### Question:
{}

### Response:

## Intent:
{}

## Knowledge Required:
{}

<think>
{}
</think>
{}"""

In [ ]:
prompt_style = """Below is an instruction that describes a mathematical task, paired with additional context information to guide the solution.
Write a response that thoroughly solves the given problem.
Before solving, develop a clear step-by-step chain of reasoning to ensure accuracy and logical coherence.

### Instruction:
You are a mathematics expert with advanced knowledge in mathematical reasoning, problem-solving, and proof techniques. You think outloud and consider various aspects before giving any concrete answers.

### Question:
{}

### Response:
<think>{}"""

In [ ]:
max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

lora_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.2.12: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

==((====))==  Unsloth 2025.2.12: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.2.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which the model when to stop generating text during training
def split_response(example):
    # Split the response on </think>
    parts = example['response'].split('</think>')

    # Get the thinking part (remove <think> tag and strip whitespace)
    thinking = parts[0].replace('<think>', '').strip()

    # Get the response part (everything after </think>, or empty string if nothing after)
    response = parts[1].strip() if len(parts) > 1 else ""

    # Return new columns
    return {
        'thinking': thinking,
        'response': response  # This will override the original response column
    }

# Apply the transformation
filtered_ds = filtered_ds.map(split_response)

Map:   0%|          | 0/1187 [00:00<?, ? examples/s]

In [ ]:
def formatting_prompts_func(examples):
    questions = examples["instruction"]
    intent = examples["intent"]
    knowledge = examples["knowledge"]
    thinking = examples["thinking"]
    response = examples["response"]

    texts = []

    for q, i, k, t, r in zip(questions, intent, knowledge, thinking, response):
        text = train_prompt_style.format(q, i, k, t, r) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}


In [ ]:
filtered_ds = filtered_ds.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/1187 [00:00<?, ? examples/s]

In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter

def split_dataset(dataset, test_size=0.1, val_size=0.1, random_state=42):
    """
    Split a dataset into train, test, and validation sets while stratifying by difficulty.
    Only keeps the 'text' column in the resulting datasets.

    Args:
        dataset: HuggingFace dataset
        test_size: proportion of data for test set
        val_size: proportion of data for validation set
        random_state: random seed for reproducibility

    Returns:
        train_dataset, test_dataset, val_dataset
    """
    # Get initial difficulty distribution
    difficulty_counts = Counter(dataset['difficulty'])
    print("Original distribution:")
    for difficulty, count in difficulty_counts.items():
        print(f"{difficulty}: {count}")

    # Create indices for splitting
    indices = list(range(len(dataset)))
    difficulties = dataset['difficulty']

    # First split: separate test set
    train_val_indices, test_indices = train_test_split(
        indices,
        test_size=test_size,
        stratify=difficulties,
        random_state=random_state
    )

    # Second split: separate validation set from training set
    # Adjust val_size to account for reduced dataset size
    adjusted_val_size = val_size / (1 - test_size)
    train_indices, val_indices = train_test_split(
        train_val_indices,
        test_size=adjusted_val_size,
        stratify=[difficulties[i] for i in train_val_indices],
        random_state=random_state
    )

    # Create the datasets with all columns first for distribution checking
    train_ds_full = dataset.select(train_indices)
    test_ds_full = dataset.select(test_indices)
    val_ds_full = dataset.select(val_indices)

    # Print distributions
    print("\nTrain set distribution:")
    train_counts = Counter(train_ds_full['difficulty'])
    for difficulty, count in train_counts.items():
        print(f"{difficulty}: {count}")

    print("\nTest set distribution:")
    test_counts = Counter(test_ds_full['difficulty'])
    for difficulty, count in test_counts.items():
        print(f"{difficulty}: {count}")

    print("\nValidation set distribution:")
    val_counts = Counter(val_ds_full['difficulty'])
    for difficulty, count in val_counts.items():
        print(f"{difficulty}: {count}")

    # Create the final datasets with only the 'text' column
    train_ds = train_ds_full.remove_columns(
        [col for col in dataset.column_names if col != 'text']
    )
    test_ds = test_ds_full
    val_ds = val_ds_full.remove_columns(
        [col for col in dataset.column_names if col != 'text']
    )

    # Print final sizes
    print(f"\nFinal sizes:")
    print(f"Train set size: {len(train_ds)}")
    print(f"Test set size: {len(test_ds)}")
    print(f"Validation set size: {len(val_ds)}")

    return train_ds, test_ds, val_ds

In [ ]:
_, test_ds, _ = split_dataset(filtered_ds)

Original distribution:
easy: 243
medium: 879
very easy: 17
hard: 48

Train set distribution:
medium: 703
easy: 195
hard: 38
very easy: 13

Test set distribution:
medium: 88
easy: 24
hard: 5
very easy: 2

Validation set distribution:
easy: 24
medium: 88
very easy: 2
hard: 5

Final sizes:
Train set size: 949
Test set size: 119
Validation set size: 119


In [ ]:
model.num_parameters(), lora_model.num_parameters()

(1777088000, 1786320384)

In [ ]:
questions = test_ds['instruction']
questions = [prompt_style.format(x, "") for x in questions]
batch_inputs = tokenizer(questions, return_tensors="pt", padding=True).to("cuda")

In [ ]:
model.training, lora_model.training

(False, True)

In [ ]:
FastLanguageModel.for_inference(model)
FastLanguageModel.for_inference(lora_model)
model.num_parameters(), lora_model.num_parameters()

(1777088000, 1786320384)

In [ ]:
model.training, lora_model.training

(False, False)

In [ ]:
instructions = test_ds['instruction']
difficulty = test_ds['difficulty']

In [ ]:
hard_ids = [i for i, x in enumerate(difficulty) if x == 'hard']

In [ ]:
hard_ids

[24, 28, 31, 69, 111]

In [ ]:
hard_instructions = [instructions[i] for i in hard_ids]

In [ ]:
hard_instructions_tokenized = [prompt_style.format(x, "") for x in hard_instructions]

In [ ]:
inputs = tokenizer(hard_instructions_tokenized, return_tensors="pt", padding=True, truncation=True).to("cuda")

In [ ]:
finetuned_outputs = []

for instruction in hard_instructions_tokenized:
  inputs = tokenizer(instruction, return_tensors="pt").to("cuda")
  outputs = lora_model.generate(
      input_ids=inputs.input_ids,
      attention_mask=inputs.attention_mask,
      max_new_tokens=4096,
      use_cache=True,
  )

  decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  finetuned_outputs.append(decoded_output)

In [ ]:
normal_outputs = []

for instruction in hard_instructions_tokenized:
  inputs = tokenizer(instruction, return_tensors="pt").to("cuda")
  outputs = model.generate(
      input_ids=inputs.input_ids,
      attention_mask=inputs.attention_mask,
      max_new_tokens=4096,
      use_cache=True,
  )

  decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  normal_outputs.append(decoded_output)

In [ ]:
len(normal_outputs), len(finetuned_outputs)

(5, 5)

In [ ]:
responses = [test_ds['response'][i] for i in hard_ids]

In [ ]:
type(finetuned_outputs), type(normal_outputs), type(responses)

(list, list, list)

In [ ]:
for tuned, normal, response in zip(finetuned_outputs, normal_outputs, responses):
  print('--' * 10 + 'Original Response' + '--' * 10)
  print(f'{"".join(response.split(".")[-3:])}')
  print('--' * 10 + 'Finetuned Response' + '--' * 10)
  print(f'{"".join(tuned[0].split(".")[-3:])}')
  print('--' * 10 + 'Normal Response' + '--' * 10)
  print(f'{"".join(normal[0].split(".")[-3:])}')
  print('--' * 100)

--------------------Original Response--------------------
, \(\epsilon < -1\)), the term oscillates and does not converge

Thus, the limit of \(a_n\) as \(n\) approaches infinity is:

- Infinity if \(\epsilon > 0\)
- \(a_1\) if \(\epsilon = 0\)
- 0 if \(-1 < \epsilon < 0\)
- Does not exist if \(\epsilon \leq -1\)

Therefore, the range for \(\epsilon\) such that the limit is infinity is all positive real numbers

\[
\boxed{\epsilon > 0}
\]
--------------------Finetuned Response--------------------
 The recursive formula can be expanded to:

$$a_n = a_1 \cdot (1 + \epsilon)^{n-1}$$

As \(n\) approaches infinity, \((1 + \epsilon)^{n-1}\) will approach infinity if \(1 + \epsilon > 1\), which is true when \(\epsilon > 0\)

Therefore, the range for \(\epsilon\) is \(\boxed{\epsilon > 0}\)
--------------------Normal Response--------------------


**Final Answer:**
- The range for ε is \(0 < \epsilon < 1\)
- As a result, the limit of the sequence as n approaches infinity is infinity

$\boxed{0

In [ ]:
import pickle
import json

# Create a dictionary with your data
data = {
    'finetuned_outputs': finetuned_outputs,
    'normal_outputs': normal_outputs,
    'responses': responses
}

# Save as pickle
with open('drive/MyDrive/model_outputs.pkl', 'wb') as f:
    pickle.dump(data, f)

# Save as JSON for readable format
with open('drive/MyDrive/model_outputs.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print('Data saved successfully!')

Data saved successfully!


In [ ]:
# from torch.utils.data import DataLoader

# def create_dataloader(dataset, batch_size=4, field='instruction'):

#     return DataLoader(
#         dataset[field],
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=2,
#         pin_memory=True
#     )

In [ ]:
# dataloader = create_dataloader(test_ds, batch_size=4)

In [ ]:
# from tqdm import tqdm

# def run_inference(model, tokenizer, dataloader, device="cuda"):
#     all_responses = []

#     with torch.no_grad():
#         # Wrap dataloader with tqdm
#         progress_bar = tqdm(
#             dataloader,
#             desc="Running inference",
#             total=len(dataloader),
#             unit="batch"
#         )

#         for batch in progress_bar:
#             inputs = tokenizer(
#                 batch,
#                 return_tensors="pt",
#                 padding=True,
#                 truncation=True,
#                 max_length=4096
#             ).to(device)

#             outputs = model.generate(
#                 input_ids=inputs.input_ids,
#                 attention_mask=inputs.attention_mask,
#                 max_new_tokens=4096,
#                 use_cache=True,
#             )

#             responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#             all_responses.extend(responses)

#             # Clear GPU memory
#             del inputs, outputs
#             # torch.cuda.empty_cache()

#             # Optionally update progress bar description with current batch size
#             progress_bar.set_description(f"Processing batch (size: {len(batch)})")

#     return all_responses

In [ ]:
# %%time
# normal_responses = run_inference(model, tokenizer, dataloader)

In [ ]:
# %%time
# lora_responses = run_inference(lora_model, tokenizer, dataloader)